In [1]:
%load_ext autoreload
from google.auth import default

credentials, project = default()

In [3]:
%autoreload 2
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)
from dotenv import load_dotenv

load_dotenv()

from src.config_loader import config
from src.gcs_storage import read_from_gcs, upload_to_gcs, get_wiktionary_cache_path
from src.wiktionary import purge_word_link_cache

print(config.TARGET_LANGUAGE_NAME)
COLLECTION = "LM2000"

Swedish


# Wiktionary Links

```raw
These are stored as a space separated string of either the words or a <a href> <\a> item.

Within the phrase dictionary they are stored with the key 'wiktionary_link'

phrase_key : {'english' : phrase key,
            'french' : phrase key,
            'wiktionary_links' : <a href>phrase</a> key}```

We store the wiktionary links as a cache so they can be reused in, e.g. the stories

In [3]:
# get current translated JSON file
from src.gcs_storage import get_translated_phrases_path

phrase_translations = read_from_gcs(
    bucket_name=config.GCS_PRIVATE_BUCKET,
    file_path=get_translated_phrases_path(collection=COLLECTION),
)

In [4]:
word_link_cache = read_from_gcs(
    config.GCS_PRIVATE_BUCKET, file_path=get_wiktionary_cache_path()
)

In [ ]:
get_wiktionary_cache_path()

In [ ]:
from src.wiktionary import add_wiktionary_links

phrase_translations, word_link_cache = add_wiktionary_links(
    phrase_translations, word_link_cache, overwrite=True
)

In [ ]:
upload_to_gcs(
    obj=phrase_translations,
    bucket_name=config.GCS_PRIVATE_BUCKET,
    file_name=get_translated_phrases_path(collection=COLLECTION),
)

In [ ]:
upload_to_gcs(
    word_link_cache,
    bucket_name=config.GCS_PRIVATE_BUCKET,
    file_name=get_wiktionary_cache_path(),
)

In [ ]:
phrase_translations

In [ ]:
word_link_cache

# PUrge cache

In [5]:
clean_cache = purge_word_link_cache()

In [6]:
upload_to_gcs(
    clean_cache,
    bucket_name=config.GCS_PRIVATE_BUCKET,
    file_name=get_wiktionary_cache_path(),
)

'gs://audio-language-trainer-private-content/resources/swedish/swedish_wiktionary_cache.json'